In [0]:
from __future__ import absolute_import, division, print_function

!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

### Loading and Scaling the Data

In [0]:
datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

In [0]:
BUFFER_SIZE = 10 # Use a much larger value for real code. 
BATCH_SIZE = 64
NUM_EPOCHS = 5

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    
    return image, label

train_data = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE).take(5)
test_data = mnist_test.map(scale).batch(BATCH_SIZE).take(5)


### Model Architecture

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu',
                           input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Training


In [0]:

metrics_names = model.metrics_names
NUM_EPOCHS=5

for epoch in range(NUM_EPOCHS):
  #Reset the metric accumulators
  model.reset_metrics()

    for image_batch, label_batch in train_data:
    result = model.train_on_batch(image_batch, label_batch)
    print("train: ",
      "{}: {:.3f}".format(metrics_names[0], result[0]),
      "{}: {:.3f}".format(metrics_names[1], result[1]))
    for image_batch, label_batch in test_data:
    result = model.test_on_batch(image_batch, label_batch, 
                             # return accumulated metrics
                             reset_metrics=False)
    print("\neval: ",
        "{}: {:.3f}".format(metrics_names[0], result[0]),
        "{}: {:.3f}".format(metrics_names[1], result[1]))


PS: I ran this in Google Labs and downloaded the notebook file. For some reason the output of the training was not printed in the downloded notebook. Thus im adding it as markdown text below just for reference.

train:  loss: 2.638 accuracy: 0.062
train:  loss: 1.756 accuracy: 0.469
train:  loss: 1.299 accuracy: 0.547
train:  loss: 1.198 accuracy: 0.641
train:  loss: 1.234 accuracy: 0.594

eval:  loss: 2.067 accuracy: 0.578
train:  loss: 0.508 accuracy: 0.906
train:  loss: 0.453 accuracy: 0.891
train:  loss: 0.489 accuracy: 0.891
train:  loss: 0.415 accuracy: 0.938
train:  loss: 0.505 accuracy: 0.875

eval:  loss: 1.913 accuracy: 0.591
train:  loss: 0.245 accuracy: 0.922
train:  loss: 0.304 accuracy: 0.938
train:  loss: 0.287 accuracy: 0.938
train:  loss: 0.212 accuracy: 0.969
train:  loss: 0.293 accuracy: 0.953

eval:  loss: 1.815 accuracy: 0.641
train:  loss: 0.202 accuracy: 0.969
train:  loss: 0.188 accuracy: 0.953
train:  loss: 0.233 accuracy: 0.953
train:  loss: 0.156 accuracy: 1.000
train:  loss: 0.213 accuracy: 0.969

eval:  loss: 1.754 accuracy: 0.672
train:  loss: 0.138 accuracy: 0.984
train:  loss: 0.141 accuracy: 0.969
train:  loss: 0.110 accuracy: 1.000
train:  loss: 0.142 accuracy: 0.984
train:  loss: 0.164 accuracy: 1.000

eval:  loss: 1.717 accuracy: 0.734

### Saving the model for tensorflow serving

In [0]:
tf.saved_model.save(model, "/serving/models/1/")